<a href="https://colab.research.google.com/github/Allzer/Machine_learning/blob/main/DL_%D0%9C%D0%BE%D1%8F_%D1%81%D0%B2%D1%91%D1%80%D1%82%D0%BE%D1%87%D0%BD%D0%B0%D1%8F_%D0%BD%D0%B5%D0%B9%D1%80%D0%BE%D0%BD%D0%BA%D0%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Задача. Создать нейронную сеть, которая будет классифицировать изображения с точностью больше 60%

In [26]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import matplotlib
from tqdm import tqdm
import scipy.io as sio
import os
from numpy.random.mtrand import shuffle
from tqdm import tqdm_notebook

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

%matplotlib inline

In [27]:
transforms = transforms.Compose([transforms.ToTensor()]) #позволяет делать преобразования с нашей выборкой (преобразуем картинку в тензор) , transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
trainset = torchvision.datasets.SVHN(root='drive/MyDrive/data', split='extra', transform=transforms, target_transform=None, download=True)
trainloader = torch.utils.data.DataLoader(trainset,batch_size = 4,shuffle = True, num_workers=2)

valset = torchvision.datasets.SVHN(root='drive/MyDrive/data',split='train', transform=transforms, target_transform=None, download=True)
valloader = torch.utils.data.DataLoader(trainset,batch_size = 4,shuffle = None, num_workers=2)

testset = torchvision.datasets.SVHN(root='drive/MyDrive/data', split='test', transform=transforms, target_transform=None, download=True)
testloader = torch.utils.data.DataLoader(testset, batch_size = 4,shuffle = False, num_workers=2)

Using downloaded and verified file: drive/MyDrive/data/extra_32x32.mat
Using downloaded and verified file: drive/MyDrive/data/train_32x32.mat
Using downloaded and verified file: drive/MyDrive/data/test_32x32.mat


In [28]:
classes = tuple(str(i) for i in range(10))

In [29]:
class simpleNS(nn.Module):
  def __init__(self):
    super(simpleNS, self).__init__()
    self.conv1 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5)
    self.pool = nn.MaxPool2d(kernel_size=2,stride=2)
    self.conv2 = nn.Conv2d(in_channels=6, out_channels=20, kernel_size=5)

    self.fc1 = nn.Linear(4*25*5,500)
    self.fc2 = nn.Linear(500, 250)
    self.fc3 = nn.Linear(250,50)
    self.fc4 = nn.Linear(50,10)

  def forward(self,x):
    x = self.pool(F.relu(self.conv1(x)))
    x = self.pool(F.relu(self.conv2(x)))
    #print(x.shape)
    x = x.view(-1,4*25*5)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = F.relu(self.fc3(x))
    x = self.fc4(x)
    return x

In [ ]:
net = simpleNS()
batch = next(iter(trainloader))
net.forward(torch.FloatTensor(batch[0]))

In [ ]:
loss_fn = torch.nn.CrossEntropyLoss()
LR = 1e-4
optimizer = torch.optim.Adam(net.parameters(),lr=LR)
losses = []

for epoch in tqdm_notebook(range(3)):
  running_loss = 0.0

  for i,batch in enumerate(tqdm_notebook(trainloader)):
    x_batch, y_batch = batch
    optimizer.zero_grad()
    y_pred = net(x_batch)
    loss = loss_fn(y_pred,y_batch)
    loss.backward()
    optimizer.step()
    running_loss += loss.item()

    if i%2000 == 1999:
      print('[%d, %5d] loss: %.3f' %(epoch+1, i+1, running_loss/2000))
      losses.append(running_loss)
      running_loss = 0.0



plt.figure(figsize=(10,7))
plt.plot(np.arange(len(losses)),losses)
plt.show()

In [32]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))

with torch.no_grad():
  for data in testloader:
    images, labels = data
    y_pred = net(images)
    _, predicted = torch.max(y_pred,1)
    c = (predicted == labels).squeeze()
    for i in range(4):
      label = labels[i]
      class_correct[label] += c[i].item()
      class_total[label] +=1

for i in range(10):
  print('Accuracy of %5s : %2d %%' %(classes[i], 100*class_correct[i]/class_total[i]))


Accuracy of     0 : 93 %
Accuracy of     1 : 94 %
Accuracy of     2 : 94 %
Accuracy of     3 : 92 %
Accuracy of     4 : 95 %
Accuracy of     5 : 84 %
Accuracy of     6 : 92 %
Accuracy of     7 : 93 %
Accuracy of     8 : 88 %
Accuracy of     9 : 90 %
